<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/hallucination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - hallucination
For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold).

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/ColabDesign.git
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params
  for W in openfold_model_ptm_1 openfold_model_ptm_2 openfold_model_no_templ_ptm_1
  do wget -qnc https://files.ipd.uw.edu/krypton/openfold/${W}.npz -P params; done
fi

In [ ]:
#@title #import libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
#@title Hallucination Options
length =  100#@param {type:"integer"}
copies =  1
#@markdown ####Weights
#@markdown - Minimizing `pae` or maximizing `plddt` often results in a single helix.
#@markdown To avoid this, we start with a random sequence and instead try to optimize 
#@markdown defined `num`ber of `con`tacts per position. 
pae = 0.1 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
plddt = 0.1 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
helix = 0.0 
con = 1.0 #@param ["0.01", "0.1", "0.5", "1.0"] {type:"raw"}
#@markdown ####Contact Definition
#@markdown - The contact definition is based on Cb-Cb diststance `cutoff`. To avoid 
#@markdown biasing towards helical contact, only contacts with sequence seperation > 
#@markdown `seqsep` are considered.


seqsep = 9 #@param ["0","5","9"] {type:"raw"}
cutoff = "14" #@param ["8", "14", "max"]
num = "2" #@param ["1", "2", "4", "8", "max"]
binary = False #@param {type:"boolean"}
if cutoff == "max": cutoff = 21.6875
if num == "max": num = length

opt = {"con":{"seqsep":int(seqsep),"cutoff":float(cutoff),"num":int(num),
              "binary":binary}}
weights = {"con":float(con),"helix":float(helix),
           "pae":float(pae),"plddt":float(plddt)}

if "model" not in dir() or model._len != length or model._copies != copies:
  clear_mem()
  model = mk_afdesign_model(protocol="hallucination")
  model.prep_inputs(length=length, copies=copies)

# pre-design with gumbel initialization and softmax activation
model.restart(mode="gumbel",opt=opt,weights=weights)
model.design_soft(50)

# three stage design  
model.restart(seq=model.aux["seq"]["pseudo"],
              keep_history=True, opt=opt,weights=weights)
model.design_3stage(50,50,10)

In [ ]:
#@markdown ## display hallucinated protein {run: "auto"}
color = "pLDDT" #@param ["chain", "pLDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb(show_sidechains=show_sidechains,
                   show_mainchains=show_mainchains,
                   color=color)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()